In [65]:
import pandas as pd
import numpy as np
import tensorflow as tf
import opensmile

In [66]:
sample_dir = './After Splitting/ReadText/PD/ID04_pd_2_0_1_3.wav'
# sample_dir = './After Splitting/ReadText/HC/ID05_hc_0_0_0_3.wav'
model_path = './full_model_testing/models/model_val_acc_3.h5'

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv01b,
    feature_level=opensmile.FeatureLevel.Functionals)

df = smile.process_file(sample_dir)
df.reset_index(drop=True, inplace=True)
df

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,22.457399,0.140746,20.66666,22.919146,24.585247,3.918587,332.168976,300.17627,67.162125,65.784363,...,-0.029331,-0.010663,0.062843,3.71732,2.662843,0.208308,0.166425,0.164138,0.283877,-38.275909


In [67]:
def sample_df_to_dataset(dataframe, shuffle=True, batch_size=20):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

ds = sample_df_to_dataset(df)

In [68]:
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
F0semitoneFrom27.5Hz_sma3nz_ame [(None, 1)]          0                                            
__________________________________________________________________________________________________
F0semitoneFrom27.5Hz_sma3nz_std [(None, 1)]          0                                            
__________________________________________________________________________________________________
F0semitoneFrom27.5Hz_sma3nz_per [(None, 1)]          0                                            
__________________________________________________________________________________________________
F0semitoneFrom27.5Hz_sma3nz_per [(None, 1)]          0                                            
_______________________________________________________________________________________

In [69]:
y_pred = model.predict(ds)
y_pred.reshape(-1)
y_pred[y_pred <= 0.5] = 0.
y_pred[y_pred > 0.5] = 1.

In [70]:
# 0 for healthy
# 1 for Parkinson's disease
int(y_pred)

1